In [ ]:
import torch
from torch.autograd import Variable
from scipy.stats import multivariate_normal
from sklearn.metrics.pairwise import rbf_kernel
import numpy as np

# 化简后的 $\frac{\nabla_{x}p(x)}{p(x)}$ 。其中，p(x)是标准正态分布。

In [1]:
def grad_log_density(x, mu, sigma,  z_optim=False):
    dtype = torch.FloatTensor

    mu = Variable(torch.Tensor(mu).type(dtype), requires_grad=z_optim)
    sigma = Variable(torch.Tensor(sigma).type(dtype), requires_grad=False)
    x = Variable(torch.Tensor(x).type(dtype), requires_grad=True)

    y = (-1 / 2) * torch.dot(x - mu, torch.inverse(sigma).mv(x - mu))

    y.backward()

    if z_optim:
        return dict(x_grad=x.grad, mu_grad=mu.grad)

    return x.grad.data.numpy()

In [ ]:
def compute_sq(i):
    return grad_log_density(x[i],mu,sigma)

# 计算 $\nabla_{x^{'}}k(x,x^{'})$

In [ ]:
def grad_k_j(i, j):
    dtype = torch.FloatTensor
    i = Variable(torch.Tensor(i).type(dtype), requires_grad=True)
    j = Variable(torch.Tensor(j).type(dtype), requires_grad=True)

    y = torch.exp(-1 / 2 * torch.square(torch.norm(i - j)))

    y.backward()
    return j.grad.data.numpy()

# 计算 $\nabla_{x}k(x,x^{'})$

In [ ]:
def grad_k_i(i,j):
    dtype = torch.FloatTensor
    i = Variable(torch.Tensor(i).type(dtype), requires_grad=True)
    j = Variable(torch.Tensor(j).type(dtype), requires_grad=True)

    y = torch.exp(-1/2*torch.square(torch.norm(i-j)))

    y.backward()

    return i.grad.data.numpy()

# 计算 $\nabla_{x,x^{'}}k(x,x^{'})$ 猜的不知道对不对

In [ ]:
def grad_k_ij(i,j):
    dtype = torch.FloatTensor
    i = Variable(torch.Tensor(i).type(dtype), requires_grad=True)
    j = Variable(torch.Tensor(j).type(dtype), requires_grad=True)

    y = torch.exp(-1/2*torch.square(torch.norm(i-j)))

    y.backward()

    ans = i.grad.data.numpy().reshape(-1,1).dot(j.grad.data.numpy().reshape(1,-1))

    return np.trace(ans)

# 生成数据

In [ ]:
N = 20
mu=np.array([1, 0., 0.])
sigma=np.eye(3)
p_data = multivariate_normal.rvs(mean=mu, cov=sigma, size=N)


x = p_data

# 计算$u_{q}(x,x^{'})$

In [ ]:
def compute_uq(i,j):
    uq = np.array(0)

    sq_i = compute_sq(i).reshape(-1,1)
    sq_j = compute_sq(j).reshape(-1,1)
    k_ij = rbf_kernel(x[i].reshape(-1,1),x[j].reshape(-1,1),gamma=1.0)

    uq = uq + sq_i.T.dot(k_ij).dot(sq_i)
    uq = uq + sq_i.T.dot(grad_k_j(x[i],x[j]))

    uq = uq + grad_k_i(x[i],x[j]).T.dot(sq_j)

    uq = uq + grad_k_ij(x[i],x[j])
    return uq

# 计算 $\hat{S}_{u}(p,q)$

In [ ]:
Supq = np.array(0)
for i in range(N):
    for j in range(N):
        if i==j:
            continue
        Supq = Supq + compute_uq(i,j)
print('Supq',Supq/(N*(N-1)))

# 计算 $\hat{S}^{*}_{u}(p,q)$   不太明白

In [ ]:
num = N
temp = [1/num]*num
a = np.random.multinomial(n=num, pvals=temp, size = 1).reshape(-1,1)

w = []

for i in range(a.shape[0]):
    for j in range(a[i][0]):
        w.append(np.random.normal())

w = np.array(w)
w = (w-1/num).reshape(-1,1)


a = np.random.multinomial(n=num, pvals=temp, size = 1).reshape(-1,1)



S_xing_upq = 0
for i in range(N):
    for j in range(N):
        if i == j:
            continue
        S_xing_upq = S_xing_upq + w[i]*w[j]*compute_uq(i,j)

print("S_xing_upq",S_xing_upq)
